Tras completar la primera etapa de limpieza y estandarización de la base de datos de facturación en Power BI, DataVision Analytics asigna al candidato la segunda fase del proyecto para TechCore.
En esta nueva instancia, el objetivo es diseñar e implementar el modelo relacional que permitirá integrar la información de ventas con las demás entidades del negocio —productos, clientes, sucursales y vendedores— asegurando coherencia, integridad y trazabilidad de los datos.

El candidato debe comenzar cargando en Python el dataset limpio exportado desde Power Query en el avance anterior (ventasTransformed.csv). A partir de esa base, debe identificar las entidades principales y sus relaciones, definir las claves primarias y foráneas y construir las tablas que conformarán la estructura del modelo: Facturas, DetalleFacturas, Productos, Clientes, Sucursales, Ciudades y Vendedores.
Cada tabla debe diseñarse con los atributos necesarios y las relaciones adecuadas para mantener la integridad referencial entre los datos; por ejemplo, asegurando que todo FacturaID presente en DetalleFacturas exista en Facturas, y que cada ProductoID o SucursalID referenciado corresponda a un registro válido.

El trabajo implica traducir la estructura conceptual del modelo entidad–relación a un entorno de análisis en Python, utilizando librerías como pandas y numpy, y opcionalmente SQLite para persistir las tablas. Una vez validadas las relaciones, se debe generar un archivo de salida (modeloVentas.xlsx) que contenga todas las tablas listas para su importación en Power BI.

Durante esta etapa, el candidato también deberá realizar verificaciones de consistencia y control de calidad de los datos, comprobando que no existan registros huérfanos ni valores incongruentes. Además, se solicita elaborar reportes exploratorios rápidos en Python —por ejemplo, el total de ventas por marca y el top 10 de productos más vendidos— para confirmar la correcta estructuración del modelo antes de integrarlo en Power BI.

Finalmente, el modelo completo deberá representarse mediante un diagrama entidad–relación detallado, donde se visualicen las tablas creadas, las claves primarias y foráneas, y la cardinalidad de cada vínculo.

El notebook con todo el proceso deberá guardarse como Avance_2_Modelo_Relacional.ipynb, y el archivo generado en Excel, modeloVentas.xlsx, servirá como base de datos definitiva para el desarrollo del dashboard en la siguiente fase.

In [38]:
# Importar librerias Necesarias
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

### Importacion del dataset limpio

Se importa el dataset limpio generado en Power BI para comenzar con el Modelado Relacional



In [24]:
df = pd.read_csv('/Users/juanv/Documents/GitHub/An-lisis-y-visualizaci-n-de-datos-de-ventas-del-sector-retail/Ventas_Transformada.csv', sep=';')
df.head(30)

,VentaID,FechaVenta,HoraVenta,SucursalNombre,CiudadSucursal,VendedorNombre,ClienteNombre,GeneroCliente,EdadCliente,EmailCliente,...,CantidadProducto2,PrecioUnitarioProducto2,SubtotalProducto2,NombreProducto3,MarcaProducto3,CantidadProducto3,PrecioUnitarioProducto3,SubtotalProducto3,DescuentoVenta,TotalVenta
0,1,31/12/2015,5:42:43 a. m.,Techcore Pereira,Pereira,Amílcar Ortega-Alberto,Bienvenida Nebot-Fiol,F,37,bienvenida19@hotmail.com,...,2.0,8000000.0,16000000.0,Dell Latitude 7420,Dell,1.0,5600000.0,5600000.0,0,31200000
1,2,23/03/2019,7:03:21 p. m.,Techcore Medellín #1,Medellín,Ana Sofía Llopis Blázquez,Teófila Bueno-Novoa,F,43,teofila35@gmail.com,...,1.0,6800000.0,6800000.0,NaN,NaN,NaN,NaN,NaN,0,10800000
2,3,23/12/2018,2:32:34 a. m.,Techcore Medellín #2,Medellín,Juan José Porcel-Riera,Gilberto Chamorro Catalá,M,38,gilberto57@hotmail.com,...,1.0,7200000.0,7200000.0,HP Pavilion 15,HP,1.0,3500000.0,3500000.0,0,21900000
3,4,14/11/2015,12:37:36 a. m.,Techcore Medellín #2,Medellín,Juan José Porcel-Riera,Máximo Coronado Huerta,M,30,maximo65@hotmail.com,...,2.0,4400000.0,8800000.0,NaN,NaN,NaN,NaN,NaN,0,13600000
4,5,29/11/2016,10:34:20 a. m.,Techcore Cali,Cali,Jacinta Juárez Marín,Yago Oliver,M,51,yago38@yahoo.com,...,1.0,3500000.0,3500000.0,Microsoft Surface Laptop 4,Microsoft,1.0,5200000.0,5200000.0,0,13900000
5,6,8/01/2015,10:33:55 p. m.,Techcore Cali,Cali,Clímaco Salinas Zabaleta,Conrado Patiño Tejero,M,22,conrado82@hotmail.com,...,2.0,2400000.0,4800000.0,HP Omen 16,HP,1.0,6000000.0,6000000.0,0,17600000
6,7,2/10/2022,11:56:06 a. m.,Techcore Cali,Cali,Jacinta Juárez Marín,Isabela de Jover,F,36,isabela71@gmail.com,...,2.0,4800000.0,9600000.0,NaN,NaN,NaN,NaN,NaN,15,12920000
7,8,4/06/2022,7:00:48 a. m.,Techcore Pereira,Pereira,Carina Parejo Carranza,Cristian Diez Gual,M,43,cristian12@yahoo.com,...,1.0,4400000.0,4400000.0,Dell Latitude 7420,Dell,2.0,5600000.0,11200000.0,5,23940000
8,9,14/01/2019,12:27:53 p. m.,Techcore Bogotá #2,Bogotá,Edu Juan Pedraza,Gertrudis Espinosa-Jódar,F,35,gertrudis61@gmail.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4800000
9,10,19/07/2023,2:25:30 p. m.,Techcore Bogotá #2,Bogotá,Armida Azorin Plaza,Sergio Paulino Cánovas Redondo,M,24,sergio78@hotmail.com,...,2.0,6800000.0,13600000.0,NaN,NaN,NaN,NaN,NaN,0,28000000


In [25]:
# Generados de Ids
def generate_ids(df, id_col, prefijo, start=1):
    """
    Genera IDs con prefij.

    df      : DataFrame a llamar
    id_col  : nombre de la columna a crear
    prefijo : prefijo del ID 
    start   : valor inicial (default 1) 
    """
    
    df = df.reset_index(drop=True).copy()
    df[id_col] = prefijo + (df.index + start).astype(str)
    return df

# Extraer Productos Para normalizar
def extraer_productos(df):
    """
    Extrae columnas de productos (Nombre, Marca, PrecioUnitario)
    desde un DataFrame con estructura Producto1, Producto2, Producto3...

    Retorna un DataFrame con columnas: ProductoNombre, Marca y PrecioUnitario
    """

    productos = []
    for i in [1,2,3]:
        temp = df[
            [
                f'NombreProducto{i}',
                f'MarcaProducto{i}',
                f'PrecioUnitarioProducto{i}'
            ]
        ].copy()

        temp.columns = ['ProductoNombre', 'Marca', 'PrecioUnitario']
        productos.append(temp)
        producto = pd.concat(productos, ignore_index=True)

    return producto

In [26]:
# Tabla Clientes

# Limpiar espacios en todas las columnas de texto
df['ClienteNombre'] = df['ClienteNombre'].str.strip()
df['VendedorNombre'] = df['VendedorNombre'].str.strip()
df['SucursalNombre'] = df['SucursalNombre'].str.strip()

# Tabla Clientes (con datos limpios)
clientes = df[['ClienteNombre', 'EdadCliente', 'GeneroCliente', 'TelefonoCliente', 'DireccionCliente', 'EmailCliente']].drop_duplicates()
clientes = generate_ids(clientes, 'ClienteID', 'CL-')
clientes = clientes[['ClienteID', 'ClienteNombre', 'EdadCliente', 'GeneroCliente', 'TelefonoCliente', 'DireccionCliente', 'EmailCliente']]
clientes.head()

,ClienteID,ClienteNombre,EdadCliente,GeneroCliente,TelefonoCliente,DireccionCliente,EmailCliente
0,CL-1,Bienvenida Nebot-Fiol,37,F,+34806548767,cll 52 #32-98,bienvenida19@hotmail.com
1,CL-2,Teófila Bueno-Novoa,43,F,+34947255990,cll 96 #81-94,teofila35@gmail.com
2,CL-3,Gilberto Chamorro Catalá,38,M,+34978810249,cra 28 #79-85,gilberto57@hotmail.com
3,CL-4,Máximo Coronado Huerta,30,M,+34825429634,cll 5 #89-26,maximo65@hotmail.com
4,CL-5,Yago Oliver,51,M,+34988285275,cra 32 #97-86,yago38@yahoo.com


In [27]:
# Tabla Ciudad

ciudades = df[['CiudadSucursal']].drop_duplicates()
ciudades = generate_ids(ciudades, 'CiudadID', 'CITY-')
ciudades = ciudades[['CiudadID', 'CiudadSucursal']]
ciudades.head()

,CiudadID,CiudadSucursal
0,CITY-1,Pereira
1,CITY-2,Medellín
2,CITY-3,Cali
3,CITY-4,Bogotá


In [28]:
# Tabla Sucursales

sucursales = df[['SucursalNombre', 'CiudadSucursal']].drop_duplicates().merge(ciudades, on='CiudadSucursal') # .merge() es para referenciar las tablas
sucursales = generate_ids(sucursales, 'SucursalID', 'SCL-')
sucursales = sucursales[['SucursalID', 'SucursalNombre', 'CiudadID']]
sucursales.head()

,SucursalID,SucursalNombre,CiudadID
0,SCL-1,Techcore Pereira,CITY-1
1,SCL-2,Techcore Medellín #1,CITY-2
2,SCL-3,Techcore Medellín #2,CITY-2
3,SCL-4,Techcore Cali,CITY-3
4,SCL-5,Techcore Bogotá #2,CITY-4


In [29]:
# Tabla Vendedores

vendedores = (df[['VendedorNombre', 'SucursalNombre']].drop_duplicates().merge(sucursales[['SucursalID', 'SucursalNombre']], on='SucursalNombre')) # .merge() es para referenciar las tablas
vendedores = generate_ids(vendedores, 'VendedorID', 'VE-')
vendedores = vendedores[['VendedorID', 'VendedorNombre', 'SucursalID', 'SucursalNombre']]
vendedores.head()

,VendedorID,VendedorNombre,SucursalID,SucursalNombre
0,VE-1,Amílcar Ortega-Alberto,SCL-1,Techcore Pereira
1,VE-2,Ana Sofía Llopis Blázquez,SCL-2,Techcore Medellín #1
2,VE-3,Juan José Porcel-Riera,SCL-3,Techcore Medellín #2
3,VE-4,Jacinta Juárez Marín,SCL-4,Techcore Cali
4,VE-5,Clímaco Salinas Zabaleta,SCL-4,Techcore Cali


In [30]:
# Tabla de Metodos de Pago

metodoPago = df[['MetodoPago']].drop_duplicates()
metodoPago = generate_ids(metodoPago, 'MetodoID', 'MP-')
metodoPago = metodoPago[['MetodoID', 'MetodoPago']]
metodoPago.head()

,MetodoID,MetodoPago
0,MP-1,Tarjeta Crédito
1,MP-2,Billetera Digital
2,MP-3,Tarjeta Débito
3,MP-4,Transferencia
4,MP-5,No Especificado


In [31]:
# Tabla de Productos
productos = extraer_productos(df)

productos = (productos.dropna(subset=['ProductoNombre']).drop_duplicates().reset_index(drop=True))
productos = generate_ids(productos, 'ProductoID', 'PD-')
productos = productos[['ProductoID', 'ProductoNombre', 'Marca', 'PrecioUnitario']]
productos.head()

,ProductoID,ProductoNombre,Marca,PrecioUnitario
0,PD-1,Apple MacBook Pro 16,Apple,9600000.0
1,PD-2,Acer Nitro 5,Acer,4000000.0
2,PD-3,Dell Latitude 7420,Dell,5600000.0
3,PD-4,Asus TUF Gaming A15,Asus,4800000.0
4,PD-5,HP Spectre x360,HP,5200000.0


In [32]:
producto_map = productos.set_index(['ProductoNombre', 'Marca'])['ProductoID']

# Mapear cada producto individualmente
for i in [1, 2, 3]:
    nombre_col = f'NombreProducto{i}'
    marca_col = f'MarcaProducto{i}'
    id_col = f'ProductoID{i}'
    
    # Crear tuplas de (nombre, marca) para hacer el mapeo
    df[id_col] = df.apply(
        lambda row: producto_map.get((row[nombre_col], row[marca_col])) 
        if pd.notna(row[nombre_col]) and pd.notna(row[marca_col]) 
        else None, 
        axis=1
    )

In [33]:
cliente_map = clientes.set_index('ClienteNombre')['ClienteID']
vendedor_map = vendedores.set_index('VendedorNombre')['VendedorID']
sucursal_map = sucursales.set_index('SucursalNombre')['SucursalID']
metodo_map = metodoPago.set_index('MetodoPago')['MetodoID']
ciudad_map = ciudades.set_index('CiudadSucursal')['CiudadID']

df['ClienteID']  = df['ClienteNombre'].map(cliente_map)
df['VendedorID'] = df['VendedorNombre'].map(vendedor_map)
df['SucursalID'] = df['SucursalNombre'].map(sucursal_map)
df['MetodoID']   = df['MetodoPago'].map(metodo_map)
df['CiudadID']   = df['CiudadSucursal'].map(ciudad_map)

In [34]:
# Crear tabla de facturas con todos los IDs
facturas = df[[
    'VentaID',
    'ClienteID',
    'VendedorID',
    'SucursalID',
    'MetodoID',
    'FechaVenta',
    'HoraVenta',
    'DescuentoVenta',
    'TotalVenta',
    'ProductoID1',
    'ProductoID2',
    'ProductoID3'
]].drop_duplicates().copy()

# Generar FacturaID
facturas = generate_ids(facturas, 'FacturaID', 'FAC# ')

# Reorganizar columnas
facturas = facturas[[
    'FacturaID',
    'VentaID',
    'ClienteID',
    'VendedorID',
    'SucursalID',
    'MetodoID',
    'FechaVenta',
    'HoraVenta',
    'DescuentoVenta',
    'TotalVenta',
    'ProductoID1',
    'ProductoID2',
    'ProductoID3'
]]

facturas.head()

,FacturaID,VentaID,ClienteID,VendedorID,SucursalID,MetodoID,FechaVenta,HoraVenta,DescuentoVenta,TotalVenta,ProductoID1,ProductoID2,ProductoID3
0,FAC# 1,1,CL-1,VE-1,SCL-1,MP-1,31/12/2015,5:42:43 a. m.,0,31200000,PD-1,PD-22,PD-3
1,FAC# 2,2,CL-2,VE-2,SCL-2,MP-2,23/03/2019,7:03:21 p. m.,0,10800000,PD-2,PD-21,None
2,FAC# 3,3,CL-3,VE-3,SCL-3,MP-3,23/12/2018,2:32:34 a. m.,0,21900000,PD-3,PD-9,PD-10
3,FAC# 4,4,CL-4,VE-3,SCL-3,MP-2,14/11/2015,12:37:36 a. m.,0,13600000,PD-4,PD-24,None
4,FAC# 5,5,CL-5,VE-4,SCL-4,MP-1,29/11/2016,10:34:20 a. m.,0,13900000,PD-5,PD-10,PD-38


In [35]:
df.head()

,VentaID,FechaVenta,HoraVenta,SucursalNombre,CiudadSucursal,VendedorNombre,ClienteNombre,GeneroCliente,EdadCliente,EmailCliente,...,DescuentoVenta,TotalVenta,ProductoID1,ProductoID2,ProductoID3,ClienteID,VendedorID,SucursalID,MetodoID,CiudadID
0,1,31/12/2015,5:42:43 a. m.,Techcore Pereira,Pereira,Amílcar Ortega-Alberto,Bienvenida Nebot-Fiol,F,37,bienvenida19@hotmail.com,...,0,31200000,PD-1,PD-22,PD-3,CL-1,VE-1,SCL-1,MP-1,CITY-1
1,2,23/03/2019,7:03:21 p. m.,Techcore Medellín #1,Medellín,Ana Sofía Llopis Blázquez,Teófila Bueno-Novoa,F,43,teofila35@gmail.com,...,0,10800000,PD-2,PD-21,None,CL-2,VE-2,SCL-2,MP-2,CITY-2
2,3,23/12/2018,2:32:34 a. m.,Techcore Medellín #2,Medellín,Juan José Porcel-Riera,Gilberto Chamorro Catalá,M,38,gilberto57@hotmail.com,...,0,21900000,PD-3,PD-9,PD-10,CL-3,VE-3,SCL-3,MP-3,CITY-2
3,4,14/11/2015,12:37:36 a. m.,Techcore Medellín #2,Medellín,Juan José Porcel-Riera,Máximo Coronado Huerta,M,30,maximo65@hotmail.com,...,0,13600000,PD-4,PD-24,None,CL-4,VE-3,SCL-3,MP-2,CITY-2
4,5,29/11/2016,10:34:20 a. m.,Techcore Cali,Cali,Jacinta Juárez Marín,Yago Oliver,M,51,yago38@yahoo.com,...,0,13900000,PD-5,PD-10,PD-38,CL-5,VE-4,SCL-4,MP-1,CITY-3


In [36]:
# Tabla detalle facturas
facturasDetalle = df[[
    'ProductoID1',
    'NombreProducto1',
    'CantidadProducto1',
    'SubtotalProducto1',
    'ProductoID2',
    'NombreProducto2',
    'CantidadProducto2',
    'SubtotalProducto2',
    'ProductoID3',
    'NombreProducto3',
    'CantidadProducto3',
    'SubtotalProducto3',
    'TotalVenta',
]].drop_duplicates()

facturasDetalle = generate_ids(facturasDetalle, 'DetalleID', 'DET# ')

facturasDetalle = facturasDetalle[[
    'DetalleID',
    'ProductoID1',
    'NombreProducto1',
    'CantidadProducto1',
    'SubtotalProducto1',
    'ProductoID2',
    'NombreProducto2',
    'CantidadProducto2',
    'SubtotalProducto2',
    'ProductoID3',
    'NombreProducto3',
    'CantidadProducto3',
    'SubtotalProducto3',
    'TotalVenta',
]]

facturasDetalle.head()

,DetalleID,ProductoID1,NombreProducto1,CantidadProducto1,SubtotalProducto1,ProductoID2,NombreProducto2,CantidadProducto2,SubtotalProducto2,ProductoID3,NombreProducto3,CantidadProducto3,SubtotalProducto3,TotalVenta
0,DET# 1,PD-1,Apple MacBook Pro 16,1,9600000,PD-22,MSI GS66 Stealth,2.0,16000000.0,PD-3,Dell Latitude 7420,1.0,5600000.0,31200000
1,DET# 2,PD-2,Acer Nitro 5,1,4000000,PD-21,Lenovo ThinkPad X1 Carbon,1.0,6800000.0,None,NaN,NaN,NaN,10800000
2,DET# 3,PD-3,Dell Latitude 7420,2,11200000,PD-9,Lenovo Legion 5 Pro,1.0,7200000.0,PD-10,HP Pavilion 15,1.0,3500000.0,21900000
3,DET# 4,PD-4,Asus TUF Gaming A15,1,4800000,PD-24,Lenovo Yoga 7i,2.0,8800000.0,None,NaN,NaN,NaN,13600000
4,DET# 5,PD-5,HP Spectre x360,1,5200000,PD-10,HP Pavilion 15,1.0,3500000.0,PD-38,Microsoft Surface Laptop 4,1.0,5200000.0,13900000


In [37]:
Excel_path = 'base_de_datos_relacional.xlsx'

with pd.ExcelWriter(Excel_path, engine='openpyxl') as writer:
    clientes.to_excel(writer, sheet_name='Clientes', index=False)
    vendedores.to_excel(writer, sheet_name='Vendedores', index=False)
    ciudades.to_excel(writer, sheet_name='Ciudades', index=False)
    sucursales.to_excel(writer, sheet_name='Sucursales', index=False) 
    metodoPago.to_excel(writer, sheet_name='MetodoPago', index=False)
    productos.to_excel(writer, sheet_name='Productos', index=False)
    facturas.to_excel(writer, sheet_name='Facturas', index=False)
    facturasDetalle.to_excel(writer, sheet_name='DetallesFacturas', index=False)

In [39]:
geolocator = Nominatim(user_agent="clientes_colombia")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def obtener_lat_lon_colombia(direccion):
    if pd.isna(direccion):
        return None, None
    
    try:
        location = geocode(f"{direccion}, Colombia")
        if location:
            return location.latitude, location.longitude
    except Exception:
        pass

    return None, None

clientes['Latitud'], clientes['Longitud'] = zip(*clientes['DireccionCliente'].apply(obtener_lat_lon_colombia))

clientes[['DireccionCliente', 'Latitud', 'Longitud']].head()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('cra 5 #83-90, Colombia',), **{}).
Traceback (most recent call last):
  File "c:\Users\juanv\miniconda3\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "c:\Users\juanv\miniconda3\Lib\site-packages\urllib3\connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
  File "c:\Users\juanv\miniconda3\Lib\http\client.py", line 1430, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "c:\Users\juanv\miniconda3\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^^
  File "c:\Users\juanv\miniconda3\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^
  File "c:\Users\juanv\miniconda3\Lib\socket.py", line 719, in readinto
    return self._soc

KeyboardInterrupt: 